# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valor te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [1]:
# Importar e carregar bibliotecas necessárias
import pandas as pd
import requests

In [2]:
# 1)
sinasc = pd.read_csv('SINASC_RO_2019.csv')
sinasc.head()

,ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,...,KOTELCHUCK,CONTADOR,munResStatus,munResTipo,munResNome,munResUf,munResLat,munResLon,munResAlt,munResArea
0,1,2679477.0,110001,1,19,5.0,8 a 11 anos,NaN,0.0,0.0,...,5,1,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
1,1,2679477.0,110001,1,29,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,2,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
2,1,2679477.0,110001,1,37,9.0,8 a 11 anos,513205.0,2.0,0.0,...,5,3,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
3,1,2516500.0,110001,1,30,5.0,12 anos ou mais,231205.0,0.0,0.0,...,4,4,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia,-12.13178,-61.85308,397.0,3958.273
4,1,2516500.0,110001,1,30,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,5,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025


In [3]:
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

Há 27.028 registros e não há registros duplicados.

In [4]:
# 2)
pd.set_option('display.max_rows', None) # Mostrar todas as linhas
sinasc.isnull().sum()

ORIGEM              0
CODESTAB          115
CODMUNNASC          0
LOCNASC             0
IDADEMAE            0
ESTCIVMAE         317
ESCMAE            312
CODOCUPMAE       2907
QTDFILVIVO       1573
QTDFILMORT       2098
CODMUNRES           0
GESTACAO         1232
GRAVIDEZ           79
PARTO              49
CONSULTAS           0
DTNASC              0
HORANASC           21
SEXO                4
APGAR1             96
APGAR5            103
RACACOR           647
PESO                0
IDANOMAL          591
DTCADASTRO          0
CODANOMAL       26814
NUMEROLOTE          0
VERSAOSIST          0
DTRECEBIM           0
DIFDATA             0
DTRECORIGA      27028
NATURALMAE        298
CODMUNNATU        298
CODUFNATU         298
ESCMAE2010        249
SERIESCMAE      12710
DTNASCMAE          40
RACACORMAE        661
QTDGESTANT       1212
QTDPARTNOR       1879
QTDPARTCES       1791
IDADEPAI        19421
DTULTMENST      10072
SEMAGESTAC       1238
TPMETESTIM       1238
CONSPRENAT        930
MESPRENAT 

In [5]:
# 3) 
sinasc_sel = sinasc[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']].copy()
sinasc_sel.isnull().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

In [7]:
# 4) 
vetor_bool_apgar = sinasc_sel['APGAR5'].isnull()
sinasc_sel = sinasc_sel[~vetor_bool_apgar]
sinasc_sel.head()

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0


In [8]:
# Número de linhas
num_linhas = sinasc_sel.shape[0]
print('A nova seleção possui %d linhas.\n' % num_linhas)
print(sinasc_sel.isnull().sum())

A nova seleção possui 26925 linhas.

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64


In [9]:
# 5)
pd.reset_option('display.max_rows') # voltar ao display inicial
sinasc_sel['ESTCIVMAE'].fillna(9, inplace = True)
sinasc_sel['CONSULTAS'].fillna(9, inplace = True)
sinasc_sel[['ESTCIVMAE', 'CONSULTAS']].isnull().sum() # verificar se não há mais NaN's para essas variáveis

ESTCIVMAE    0
CONSULTAS    0
dtype: int64

In [10]:
# 6) 
sinasc_sel['QTDFILVIVO'].fillna(0, inplace = True)
sinasc_sel['QTDFILVIVO'].isnull().sum() # verificar se não há mais NaN's para essa variável

0

In [11]:
# 7) 
# Verificar as variáveis que ainda contêm NaN's
sinasc_sel.isnull().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO       0
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

Para a variável ```ESCMAE```, vamos substituir os dados ausentes com as três maiores frequências relativas entre todas as categorias, assim, suporemos que a distribuição da variável é consistente entre as observações.

In [12]:
# Verificar frequência da variável ESCMAE
sinasc_sel['ESCMAE'].value_counts(normalize = True)

8 a 11 anos        0.585422
12 anos ou mais    0.216307
4 a 7 anos         0.177607
1 a 3 anos         0.019162
Nenhuma            0.001503
Name: ESCMAE, dtype: float64

In [13]:
sinasc_sel['ESCMAE'].fillna(value = '8 a 11 anos', inplace = True, limit = 31*6) # freq. de 60%
sinasc_sel['ESCMAE'].fillna(value = '12 anos ou mais', inplace = True, limit = 31*2) # freq de 20%
sinasc_sel['ESCMAE'].fillna(value = '4 a 7 anos', inplace = True, limit = 31*2) # freq de 20%

# Verificar se não há mais dados faltantes
sinasc_sel['ESCMAE'].isnull().sum()

0

Para a variável ```GESTACAO```, vamos preencher os dados faltantes com a maior frequência absoluta dentre todas as categorias, assim, suporemos que a distribuição da variável concentra-se nesse valor.

In [14]:
# Verificar frequência da variável GESTACAO
sinasc_sel['GESTACAO'].value_counts()

37 a 41 semanas        22523
32 a 36 semanas         2234
42 semanas e mais        698
28 a 31 semanas          162
22 a 27 semanas           85
Menos de 22 semanas        7
Name: GESTACAO, dtype: int64

In [15]:
sinasc_sel['GESTACAO'].fillna('37 a 41 semanas', inplace = True)

# Verificar se não há mais dados faltantes
sinasc_sel['GESTACAO'].isnull().sum()

0

Para a variável ```GRAVIDEZ```, vamos preencher os dados faltantes com o código 9, que significa "ignorado", pois são poucas as unidades de observação que apresentam valores ausentes.

In [16]:
# Verificar frequência da variável GRAVIDEZ
sinasc_sel['GRAVIDEZ'].value_counts()

Única              26356
Dupla                488
Tríplice e mais        5
Name: GRAVIDEZ, dtype: int64

In [17]:
sinasc_sel['GRAVIDEZ'].fillna(9, inplace = True)

# Verificar se não há mais dados faltantes
sinasc_sel['GRAVIDEZ'].isnull().sum()

0

In [18]:
# 8) 
sinasc_sel.loc[sinasc_sel['APGAR5'] >= 8, 'CLASSIF_APGAR5'] = 'Normal'
sinasc_sel.loc[(sinasc_sel['APGAR5'] >= 6) & (sinasc_sel['APGAR5'] <= 7), 'CLASSIF_APGAR5'] = 'Asfixia leve'
sinasc_sel.loc[(sinasc_sel['APGAR5'] >= 4) & (sinasc_sel['APGAR5'] <= 5), 'CLASSIF_APGAR5'] = 'Asfixia moderada'
sinasc_sel.loc[sinasc_sel['APGAR5'] <= 3, 'CLASSIF_APGAR5'] = 'Asfixia severa'
sinasc_sel.head()

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,CLASSIF_APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,Normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,Normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,Normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,Normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,Normal


In [19]:
# Frequência relativa de cada categoria da variável CLASSIF_APGAR5
sinasc_sel['CLASSIF_APGAR5'].value_counts(normalize = True)

Normal              0.982841
Asfixia leve        0.011885
Asfixia severa      0.002748
Asfixia moderada    0.002526
Name: CLASSIF_APGAR5, dtype: float64

In [20]:
# 9) Verificar os nomes de variáveis existentes
sinasc.columns

Index(['ORIGEM', 'CODESTAB', 'CODMUNNASC', 'LOCNASC', 'IDADEMAE', 'ESTCIVMAE',
       'ESCMAE', 'CODOCUPMAE', 'QTDFILVIVO', 'QTDFILMORT', 'CODMUNRES',
       'GESTACAO', 'GRAVIDEZ', 'PARTO', 'CONSULTAS', 'DTNASC', 'HORANASC',
       'SEXO', 'APGAR1', 'APGAR5', 'RACACOR', 'PESO', 'IDANOMAL', 'DTCADASTRO',
       'CODANOMAL', 'NUMEROLOTE', 'VERSAOSIST', 'DTRECEBIM', 'DIFDATA',
       'DTRECORIGA', 'NATURALMAE', 'CODMUNNATU', 'CODUFNATU', 'ESCMAE2010',
       'SERIESCMAE', 'DTNASCMAE', 'RACACORMAE', 'QTDGESTANT', 'QTDPARTNOR',
       'QTDPARTCES', 'IDADEPAI', 'DTULTMENST', 'SEMAGESTAC', 'TPMETESTIM',
       'CONSPRENAT', 'MESPRENAT', 'TPAPRESENT', 'STTRABPART', 'STCESPARTO',
       'TPNASCASSI', 'TPFUNCRESP', 'TPDOCRESP', 'DTDECLARAC', 'ESCMAEAGR1',
       'STDNEPIDEM', 'STDNNOVA', 'CODPAISRES', 'TPROBSON', 'PARIDADE',
       'KOTELCHUCK', 'CONTADOR', 'munResStatus', 'munResTipo', 'munResNome',
       'munResUf', 'munResLat', 'munResLon', 'munResAlt', 'munResArea'],
      dtype='object')

In [21]:
sinasc.columns = sinasc.columns.str.lower() # colocar em minúsculo
sinasc.columns

Index(['origem', 'codestab', 'codmunnasc', 'locnasc', 'idademae', 'estcivmae',
       'escmae', 'codocupmae', 'qtdfilvivo', 'qtdfilmort', 'codmunres',
       'gestacao', 'gravidez', 'parto', 'consultas', 'dtnasc', 'horanasc',
       'sexo', 'apgar1', 'apgar5', 'racacor', 'peso', 'idanomal', 'dtcadastro',
       'codanomal', 'numerolote', 'versaosist', 'dtrecebim', 'difdata',
       'dtrecoriga', 'naturalmae', 'codmunnatu', 'codufnatu', 'escmae2010',
       'seriescmae', 'dtnascmae', 'racacormae', 'qtdgestant', 'qtdpartnor',
       'qtdpartces', 'idadepai', 'dtultmenst', 'semagestac', 'tpmetestim',
       'consprenat', 'mesprenat', 'tpapresent', 'sttrabpart', 'stcesparto',
       'tpnascassi', 'tpfuncresp', 'tpdocresp', 'dtdeclarac', 'escmaeagr1',
       'stdnepidem', 'stdnnova', 'codpaisres', 'tprobson', 'paridade',
       'kotelchuck', 'contador', 'munresstatus', 'munrestipo', 'munresnome',
       'munresuf', 'munreslat', 'munreslon', 'munresalt', 'munresarea'],
      dtype='object')

In [22]:
sinasc.columns = ['origem', 'cod_estab', 'cod_mun_nasc', 'loc_nasc', 'idade_mae', 'est_civ_mae',
       'esc_mae', 'cod_ocup_mae', 'qtd_fil_vivo', 'qtd_fil_mort', 'cod_mun_res',
       'gestacao', 'gravidez', 'parto', 'consultas', 'dt_nasc', 'hora_nasc',
       'sexo', 'apgar_1', 'apgar_5', 'raca_cor', 'peso', 'id_anomal', 'dt_cadastro',
       'cod_anomal', 'numero_lote', 'versao_sist', 'dt_recebim', 'dif_data',
       'dt_rec_origa', 'natural_mae', 'cod_mun_natu', 'cod_uf_natu', 'esc_mae_2010',
       'seri_esc_mae', 'dt_nasc_mae', 'raca_cor_mae', 'qtd_gestant', 'qtd_part_nor',
       'qtd_part_ces', 'idade_pai', 'dt_ult_menst', 'sema_gestac', 'tp_met_estim',
       'cons_pre_nat', 'mes_pre_nat', 'tp_apresent', 'st_trab_part', 'st_ces_parto',
       'tp_nasc_assi', 'tp_func_resp', 'tp_doc_resp', 'dt_declarac', 'esc_mae_agr_1',
       'st_dn_epidem', 'st_dn_nova', 'cod_pais_res', 'tp_robson', 'paridade',
       'kotelchuck', 'contador', 'mun_res_status', 'mun_res_tipo', 'mun_res_nome',
       'mun_res_uf', 'mun_res_lat', 'mun_res_lon', 'mun_res_alt', 'mun_res_area']
sinasc.head()

,origem,cod_estab,cod_mun_nasc,loc_nasc,idade_mae,est_civ_mae,esc_mae,cod_ocup_mae,qtd_fil_vivo,qtd_fil_mort,...,kotelchuck,contador,mun_res_status,mun_res_tipo,mun_res_nome,mun_res_uf,mun_res_lat,mun_res_lon,mun_res_alt,mun_res_area
0,1,2679477.0,110001,1,19,5.0,8 a 11 anos,NaN,0.0,0.0,...,5,1,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
1,1,2679477.0,110001,1,29,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,2,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
2,1,2679477.0,110001,1,37,9.0,8 a 11 anos,513205.0,2.0,0.0,...,5,3,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
3,1,2516500.0,110001,1,30,5.0,12 anos ou mais,231205.0,0.0,0.0,...,4,4,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia,-12.13178,-61.85308,397.0,3958.273
4,1,2516500.0,110001,1,30,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,5,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
